**Cleaning For Data**
**Horizontally Add FIFA Data year by year**

In [1]:
import pandas as pd
import openpyxl
RAW_DATA_DIR = '../data/raw'

In [2]:
df_standard = pd.read_csv(f'{RAW_DATA_DIR}/standard_stats.csv', dtype =str )
df_shooting = pd.read_csv(f'{RAW_DATA_DIR}/shooting_stats.csv', dtype =str )
df_creation = pd.read_csv(f'{RAW_DATA_DIR}/creation_stats.csv', dtype =str )
df_defensive = pd.read_csv(f'{RAW_DATA_DIR}/defensive_stats.csv', dtype =str )
df_keeper = pd.read_csv(f'{RAW_DATA_DIR}/keeper_stats.csv', dtype =str )
df_adv_keeper = pd.read_csv(f'{RAW_DATA_DIR}/keeper__adv_stats.csv', dtype =str )

In [3]:

df_shooting = df_shooting.rename(columns={'90s': 'Playing Time_90s',
                                          'Standard_Gls': 'Performance_Gls',
                                          'Standard_PK': 'Performance_PK',
                                          'Standard_PKatt': 'Performance_PKatt'})
df_shooting = df_shooting.drop(columns=['Expected_xG', 'Expected_npxG', 'Expected_G-xG', 'Expected_np:G-xG'])

In [4]:
# Merge general w shooting, left concat, prioritize general stats
df_merged = pd.merge(df_standard, df_shooting, how='left')
df_merged = df_merged.drop(columns=['Performance_Gls', 'Performance_Ast', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK',  'Standard_FK', 'Expected_xG', 'Expected_npxG', 'Expected_xAG', 'Expected_npxG+xAG', 'Standard_SoT', 'Standard_Sh'])
df_merged['Performance_PKatt'] = df_merged['Performance_PKatt'].astype(float) / df_merged['Playing Time_90s'].astype(float)
df_merged['Performance_CrdY'] = df_merged['Performance_CrdY'].astype(float) / df_merged['Playing Time_90s'].astype(float)
df_merged['Performance_CrdR'] = df_merged['Performance_CrdR'].astype(float) / df_merged['Playing Time_90s'].astype(float)
df_merged['Progression_PrgC'] = df_merged['Progression_PrgC'].astype(float) / df_merged['Playing Time_90s'].astype(float)
df_merged['Progression_PrgP'] = df_merged['Progression_PrgP'].astype(float) / df_merged['Playing Time_90s'].astype(float)
df_merged['Progression_PrgR'] = df_merged['Progression_PrgR'].astype(float) / df_merged['Playing Time_90s'].astype(float)
df_merged.rename(columns={'Performance_PKatt': 'Per 90 Minutes_PKatt',
                          'Performance_CrdY': 'Per 90 Minutes_CrdY',
                            'Performance_CrdR': 'Per 90 Minutes_CrdR',
                            'Progression_PrgC': 'Per 90 Minutes_PrgC',
                            'Progression_PrgP': 'Per 90 Minutes_PrgP',
                            'Progression_PrgR': 'Per 90 Minutes_PrgR'}, inplace=True)
#df_merged.tail(10)

In [5]:
# Function for removing seasons 1415, 1516, 1617
def remove_early_seasons(df):
    seasons_to_remove = ['1415', '1516', '1617']
    df_filtered = df[~df['season'].isin(seasons_to_remove)]
    return df_filtered
df_merged = remove_early_seasons(df_merged)
df_creation = remove_early_seasons(df_creation)
df_defensive = remove_early_seasons(df_defensive)
df_keeper = remove_early_seasons(df_keeper)
df_adv_keeper = remove_early_seasons(df_adv_keeper)
#df_merged.head(10)

In [12]:
df_passing = pd.read_csv(f'{RAW_DATA_DIR}/passing_stats.csv', dtype =str )
df_passing = remove_early_seasons(df_passing)
df_passing = df_passing.drop(columns=['Ast', 'xAG', '90s', 'PrgP'])
df_passing['Playing Time_Min'] = df_merged['Playing Time_Min'].astype(float)
per90 = ['Total_Cmp', 'Total_Att', 'Total_TotDist', 'Total_PrgDist', 'Short_Cmp', 'Short_Att', 'Medium_Cmp', 'Medium_Att', 'Long_Cmp', 'Long_Att', 'Expected_xA', 'Expected_A-xAG', 'KP', '1/3', 'PPA', 'CrsPA']
for col in per90:
    df_passing[col] = df_passing[col].astype(float) / df_passing['Playing Time_Min'].astype(float) * 90
    df_passing.rename(columns={col: f'Per 90 Minutes_{col}'}, inplace=True)
#df_passing.head(10)

In [13]:
df_passing = df_passing.drop(columns=['Playing Time_Min'])
# Final merge with passing stats
df_merged = pd.merge(df_merged, df_passing, how='left')
#df_merged.head(10)

In [14]:
# Merged general, shooting, and passsing
#TODO: merge creation, defensive
#TODO: merge keeper, adv keeper; implement keeper features in modeling